In [17]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('../../smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.experiment.databaseObj import Database
from smodels.base.model import Model
from smodels.decomposition import decomposer
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.base import runtime
from smodels.experiment.expSMS import ExpSMS
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V

3.0.0-beta


In [18]:
BSMlist = getParticlesFromSLHA('TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha')
#BSMlist = getParticlesFromSLHA('slhaFiles_normal1/run_01_MZp_1500_gAq_4.000000E-02.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
#model.updateParticles(inputFile=slhaFile)
print(BSMlist)

[xd, xd~, y1]


In [19]:
slhaFolder = 'TRV1jj_slha'
resultsFolder = 'TRV1jj_results'
File = 'parameters.ini'
!chmod -R +rwx {slhaFolder}

In [20]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}

INFO in databaseObj.loadBinaryFile() in 549: loading binary db file /home/yoxara/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 556: Loaded database from /home/yoxara/smodels-database/db3.pcl in 5.9 secs.
INFO in modelTester.loadDatabaseResults() in 497: Including non-validated results
INFO in modelTester.testPoints() in 357: Running SModelS for 357 files with a single process. Messages will be redirected to smodels.log


In [21]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)

    data.append(dataDict)
    

In [22]:
print(len(data))

357


In [23]:
#Convert data to flat DataFrame:
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 7)
smodelsDF = json_normalize(data)
smodelsDF

,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_01_MZp_1500_gAq_4.000000E-02.slha,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",1.116829e+01,0.000000e+00,7.172288e+01,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,6.055459e+01,1.172950e+02,1.079320e+02,[TRV1jj],"[(y1, 1500.0)]",ATLAS-EXOT-2019-03,None,1.300000e+01,1.390000e+02,upperLimit,5.162589e-01,5.610439e-01,"[(y1, 0.1131)]"
1,run_83_MZp_2000_gAq_8.200000E-02.slha,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",1.140968e+01,0.000000e+00,7.108620e+01,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha/run_83_MZp_2000_gAq_8.200000E-02.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,5.967652e+01,3.215460e+01,4.719660e+01,[TRV1jj],"[(y1, 2000.0)]",ATLAS-EXOT-2019-03,None,1.300000e+01,1.390000e+02,upperLimit,1.855925e+00,1.264424e+00,"[(y1, 0.63738)]"
2,run_249_MZp_3100_gAq_9.648170E-02.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",1.166697e+00,0.000000e+00,7.109598e+00,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha/run_249_MZp_3100_gAq_9.648170E-02....,3.0.0-beta,3.0.0-beta,0.000000e+00,5.942900e+00,1.330300e+01,NaN,[TRV1jj],"[(y1, 3100.0)]",ATLAS-EXOT-2019-03,None,1.300000e+01,1.390000e+02,upperLimit,4.467338e-01,NaN,"[(y1, 1.3736)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,run_293_MZp_3400_gAq_8.200000E-02.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",4.398751e-01,0.000000e+00,2.673454e+00,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha/run_293_MZp_3400_gAq_8.200000E-02....,3.0.0-beta,3.0.0-beta,0.000000e+00,2.233579e+00,6.289730e+00,NaN,[TRV1jj],"[(y1, 3400.0)]",ATLAS-EXOT-2019-03,None,1.300000e+01,1.390000e+02,upperLimit,3.551152e-01,NaN,"[(y1, 1.0888)]"
355,run_323_MZp_1800_gAq_1.520000E-01.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",6.738647e+01,0.000000e+00,4.236547e+02,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Inp

In [24]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 5000001])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 5000001])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)


In [25]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [26]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [27]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 357


In [28]:
#Save DataFrame to pickle file:
dataDF.to_pickle('TRV1jj_data.pcl')
print(len(dataDF))
dataDF

357


,filename,mass.5000001,mass.5000521,width.5000001,width.5000521,extpar.1,extpar.2,extpar.3,extpar.4,xsec13TeV(fb).5000001,"BRs.y1.q,q","BRs.y1.c,c","BRs.y1.b,b","BRs.y1.t-,t+",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_01_MZp_1500_gAq_4.000000E-02.slha,1.500000e+03,1.000000e+04,1.131036e-01,0.000000e+00,0.000000e+00,1.500000e+00,4.000000e-02,0.000000e+00,7.172289e+01,1.688592e-01,1.688585e-01,1.688493e-01,1.557145e-01,7.172289e+01,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",1.116829e+01,0.000000e+00,7.172288e+01,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,6.055459e+01,1.172950e+02,1.079320e+02,[TRV1jj],"[(y1, 1500.0)]",ATLAS-EXOT-2019-03,None,1.300000e+01,1.390000e+02,upperLimit,5.162589e-01,5.610439e-01,"[(y1, 0.1131)]"
1,run_83_MZp_2000_gAq_8.200000E-02.slha,2.000000e+03,1.000000e+04,6.373773e-01,0.000000e+00,0.000000e+00,1.500000e+00,8.200000e-02,0.000000e+00,7.108620e+01,1.679002e-01,1.678998e-01,1.678947e-01,1.605049e-01,7.108620e+01,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",1.140968e+01,0.000000e+00,7.108620e+01,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha/run_83_MZp_2000_gAq_8.200000E-02.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,5.967652e+01,3.215460e+01,4.719660e+01,[TRV1jj],"[(y1, 2000.0)]",ATLAS-EXOT-2019-03,None,1.300000e+01,1.390000e+02,upperLimit,1.855925e+00,1.264424e+00,"[(y1, 0.63738)]"
2,run_249_MZp_3100_gAq_9.648170E-02.slha,3.100000e+03,1.000000e+04,1.373590e+00,0.000000e+00,0.000000e+00,1.500000e+00,9.648170e-02,0.000000e+00,7.109599e+00,1.671801e-01,1.671800e-01,1.671778e-01,1.641017e-01,7.109599e+00,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",1.166697e+00,0.000000e+00,7.109598e+00,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1jj_slha/run_01_MZp_1500_gAq_4.000000E-02.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1jj_slha/run_249_MZp_3100_gAq_9.648170E-02....,3.0.0-beta,3.0.0-beta,0.000000e+00,5.942900e+00,1.330300e+01,NaN,[TRV1jj],"[(y1, 3100.0)]",ATLAS-EXOT-2019-03,None,1.300000e+01,1.390000e+02,upperLimit,4.467338e-01,NaN,"[(y1, 1.3736)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,run_293_MZp_3400_gAq_8.200000E-02.slha,3.

In [29]:
dataDF['xsec13TeV(fb).5000001'] * (3*dataDF['BRs.y1.q,q'] + dataDF['BRs.y1.c,c'] + dataDF['BRs.y1.b,b'])

0     6.055459e+01
1     5.967652e+01
2     5.942900e+00
          ...     
354   2.233579e+00
355   3.562682e+02
356   1.784818e+01
Length: 357, dtype: float64

In [30]:
dataDF['ExptRes.result0.theory prediction (fb)']

0     6.055459e+01
1     5.967652e+01
2     5.942900e+00
          ...     
354   2.233579e+00
355   3.562682e+02
356   1.784818e+01
Name: ExptRes.result0.theory prediction (fb), Length: 357, dtype: float64

In [31]:
dataDF['ExptRes.result0.r']

0     5.162589e-01
1     1.855925e+00
2     4.467338e-01
          ...     
354   3.551152e-01
355   8.038688e+00
356   4.734490e-01
Name: ExptRes.result0.r, Length: 357, dtype: float64

In [32]:
dataDF['ExptRes.result0.theory prediction (fb)']/dataDF['ExptRes.result0.upper limit (fb)']

0     5.162589e-01
1     1.855925e+00
2     4.467338e-01
          ...     
354   3.551152e-01
355   8.038688e+00
356   4.734491e-01
Length: 357, dtype: float64